In [90]:
import numpy as np
import pandas as pd
import regex as re
import tensorflow as tf
import transformers
import os

from tokenizers import BertWordPieceTokenizer
from tensorflow.keras import Input, Model
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GlobalMaxPool1D, Dropout, Dense
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [54]:
df = pd.read_csv('tweet-sentiment-extraction/train.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [55]:
def basic_cleaning(text):
    text = re.sub(r'https?://www\.\S+\.com','',text)
    text = re.sub(r'[^A-Za-z|\s]','',text)
    text = re.sub(r'\*+','swear',text)                    # Capture swear words that are **** out
    return text


In [56]:
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"        #emoticons
        u"\U0001F300-\U0001F5FF"        #symbols & pictographs
        u"\U0001F680-\U0001F6FF"        #transport & map symbols
        u"\U0001F1E0-\U0001F1FF"        #flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_multiplechars(text):
    text = re.sub(r'(.)\1{3,}',r'\1', text)
    return text

In [57]:
def clean(df):
    for col in ['text']:#,'selected_text']:
        df[col] = df[col].astype(str).apply(lambda x:basic_cleaning(x))
        df[col] = df[col].astype(str).apply(lambda x:remove_emoji(x))
        df[col] = df[col].astype(str).apply(lambda x:remove_html(x))
        df[col] = df[col].astype(str).apply(lambda x:remove_multiplechars(x))
    return df

In [75]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=128):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids = []
    
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [76]:
def preprocess_news(df,stop,n=1,col='text'):
    '''Function to preprocess and create corpus'''
    new_corpus=[]
    stem=PorterStemmer()
    lem=WordNetLemmatizer()
    for text in df[col]:
        words=[w for w in word_tokenize(text) if (w not in stop)]
       
        words=[lem.lemmatize(w) for w in words if(len(w)>n)]
     
        new_corpus.append(words)
        
    new_corpus=[word for l in new_corpus for word in l]
    return new_corpus

In [77]:
df.dropna(inplace=True)
df_clean = clean(df)
df_clean_selection = df_clean.sample(frac=1)
X = df_clean_selection.text.values
y = pd.get_dummies(df_clean_selection.sentiment)

In [78]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(X))
list_tokenized_train = tokenizer.texts_to_sequences(X)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=128)

In [79]:
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")  
# Save the loaded tokenizer locally
save_path = './kaggle/working/distilbert_base_uncased/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
tokenizer.save_pretrained(save_path)
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer(
                 './kaggle/working/distilbert_base_uncased/vocab.txt', lowercase=True)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

In [80]:
X = fast_encode(df_clean_selection.text.astype(str),
                fast_tokenizer,
                maxlen=128)

In [81]:
transformer_layer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')
embedding_size = 128
input_ = Input(shape=(100,))
inp = Input(shape=(128, ))
embedding_matrix=transformer_layer.weights[0].numpy()
x = Embedding(embedding_matrix.shape[0],
              embedding_matrix.shape[1],
              embeddings_initializer=Constant(embedding_matrix),
              trainable=False)(inp)
x = Bidirectional(LSTM(50, return_sequences=True))(x)
x = Bidirectional(LSTM(25, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.5)(x)
x = Dense(50, activation='relu', kernel_regularizer='L1L2')(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='softmax')(x)
model_DistilBert = Model(inputs=[inp], outputs=x)
model_DistilBert.compile(loss='categorical_crossentropy',
                              optimizer='adam',
                              metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [82]:
model_DistilBert.fit(X, y, batch_size=32, epochs=10, validation_split=0.1)

Epoch 1/10
773/773 [==============================] - 146s 180ms/step - loss: 1.0768 - accuracy: 0.4141 - val_loss: 0.9665 - val_accuracy: 0.5022
Epoch 2/10
773/773 [==============================] - 144s 186ms/step - loss: 0.8265 - accuracy: 0.6257 - val_loss: 0.6943 - val_accuracy: 0.7143
Epoch 3/10
773/773 [==============================] - 144s 186ms/step - loss: 0.6825 - accuracy: 0.7183 - val_loss: 0.6260 - val_accuracy: 0.7464
Epoch 4/10
773/773 [==============================] - 143s 185ms/step - loss: 0.6409 - accuracy: 0.7415 - val_loss: 0.6164 - val_accuracy: 0.7471
Epoch 5/10
773/773 [==============================] - 138s 178ms/step - loss: 0.6140 - accuracy: 0.7565 - val_loss: 0.6050 - val_accuracy: 0.7434
Epoch 6/10
773/773 [==============================] - 143s 185ms/step - loss: 0.5943 - accuracy: 0.7617 - val_loss: 0.5939 - val_accuracy: 0.7529
Epoch 7/10
773/773 [==============================] - 151s 195ms/step - loss: 0.5794 - accuracy: 0.7693 - val_loss: 0.5905 -

In [83]:
df_clean_final = df_clean.sample(frac=1)
X_train = fast_encode(df_clean_selection.text.astype(str), fast_tokenizer, maxlen=128)
y_train = y

In [87]:
Adam_name = Adam(lr=0.001)
model_DistilBert.compile(loss='categorical_crossentropy',optimizer=Adam_name,metrics=['accuracy'])
history = model_DistilBert.fit(X_train,y_train,batch_size=32,epochs=10)

/Users/Joe/opt/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
859/859 [==============================] - 142s 156ms/step - loss: 0.5149 - accuracy: 0.7983
Epoch 2/10
859/859 [==============================] - 132s 154ms/step - loss: 0.4954 - accuracy: 0.8057
Epoch 3/10
859/859 [==============================] - 149s 174ms/step - loss: 0.4800 - accuracy: 0.8124
Epoch 4/10
859/859 [==============================] - 146s 170ms/step - loss: 0.4616 - accuracy: 0.8222
Epoch 5/10
859/859 [==============================] - 136s 158ms/step - loss: 0.4450 - accuracy: 0.8277
Epoch 6/10
859/859 [==============================] - 2028s 2s/step - loss: 0.4208 - accuracy: 0.8362
Epoch 7/10
859/859 [==============================] - 131s 153ms/step - loss: 0.4009 - accuracy: 0.8462
Epoch 8/10
859/859 [==============================] - 136s 159ms/step - loss: 0.3821 - accuracy: 0.8543
Epoch 9/10
859/859 [==============================] - 134s 156ms/step - loss: 0.3583 - accuracy: 0.8649
Epoch 10/10
859/859 [==============================] - 134s 156ms/

In [88]:
df_test = pd.read_csv('./tweet-sentiment-extraction/test.csv')
df_test.dropna(inplace=True)
df_clean_test = clean(df_test)
X_test = fast_encode(df_clean_test.text.values.astype(str),
                     fast_tokenizer,
                     maxlen=128)
y_test = df_clean_test.sentiment

In [91]:
y_preds = model_DistilBert.predict(X_test)
y_predictions = pd.DataFrame(y_preds,
                             columns=['negative','neutral','positive'])
y_predictions_final = y_predictions.idxmax(axis=1)
accuracy = accuracy_score(y_test,y_predictions_final)
print(f"The final model shows {accuracy:.2f} accuracy on the test set.")

111/111 [==============================] - 7s 63ms/step
The final model shows 0.75 accuracy on the test set.
